# 01. Ridge and Lasso Regression | الانحدار ريدج ولاسو

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Unit 1: All examples** - Data processing, linear regression, polynomial regression
- ✅ **Understanding of overfitting**: What happens when models are too complex
- ✅ **Basic linear algebra**: Understanding coefficients and regularization

**If you haven't completed these**, you might struggle with:
- Understanding why regularization is needed
- Knowing when to use Ridge vs Lasso
- Understanding how alpha (regularization strength) works

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is Unit 2, Example 1** - it solves the overfitting problem from polynomial regression!

**Why this example FIRST in Unit 2?**
- **Before** you can use advanced techniques, you need to solve overfitting
- **Before** you can build robust models, you need regularization
- **Before** you can handle multicollinearity, you need Ridge/Lasso

**Builds on**: 
- 📓 Unit 1, Example 4: Linear Regression (we know basic regression)
- 📓 Unit 1, Example 5: Polynomial Regression (we saw overfitting!)

**Leads to**: 
- 📓 Example 2: Cross-Validation (evaluates models properly)
- 📓 Unit 3: Classification (same regularization concepts apply)
- 📓 All ML models (regularization is universal!)

**Why this order?**
1. Ridge/Lasso solve **overfitting** (critical problem from Unit 1)
2. Ridge/Lasso teach **regularization** (essential ML concept)
3. Ridge/Lasso show **feature selection** (Lasso automatically selects features)

---

## The Story: Preventing Overfitting | القصة: منع الإفراط في التلائم

Imagine you're learning to drive. **Before** regularization, you memorize every turn on the training route perfectly, but fail on new routes (overfitting). **After** regularization, you learn general driving principles that work everywhere!

Same with machine learning: **Before** Ridge/Lasso, models memorize training data perfectly but fail on new data. **After** Ridge/Lasso, models learn general patterns that generalize well!

---

## Why Ridge and Lasso Matter | لماذا يهم ريدج ولاسو؟

Regularization prevents overfitting:
- **Ridge (L2)**: Shrinks coefficients toward zero (keeps all features)
- **Lasso (L1)**: Shrinks some coefficients to exactly zero (feature selection!)
- **Both**: Prevent overfitting by penalizing large coefficients
- **Alpha**: Controls regularization strength (higher = more regularization)

## Learning Objectives | أهداف التعلم
1. Build Ridge regression models (L2 regularization)
2. Build Lasso regression models (L1 regularization)
3. Understand the difference between Ridge and Lasso
4. Tune alpha hyperparameter
5. Compare regularized models with linear regression
6. Understand when to use each method

In [ ]:
# Step 1: Import necessary libraries
# These libraries help us build regularized regression models

import pandas as pd  # For data manipulation
import numpy as np   # For numerical operations
import matplotlib.pyplot as plt  # For visualizations
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.linear_model import (
    LinearRegression,  # Baseline model (no regularization)
    Ridge,             # L2 regularization (shrinks coefficients)
    Lasso              # L1 regularization (shrinks + feature selection)
)
from sklearn.preprocessing import StandardScaler  # Important! Regularization needs scaled features
from sklearn.metrics import mean_squared_error, r2_score  # For evaluation

print("✅ Libraries imported successfully!")
print("\n📚 What each model does:")
print("   - LinearRegression: No regularization (baseline)")
print("   - Ridge: L2 regularization (keeps all features, shrinks coefficients)")
print("   - Lasso: L1 regularization (removes some features, shrinks others)")
print("   - StandardScaler: CRITICAL! Regularization requires scaled features!")

## Part 1: Setting the Scene | الجزء الأول: إعداد المشهد

**BEFORE**: We saw overfitting in polynomial regression - models that fit training data too well but fail on new data.

**AFTER**: We'll use Ridge and Lasso regularization to prevent overfitting by penalizing large coefficients!

**Why this matters**: Overfitting is the #1 problem in ML. Regularization is the #1 solution!

---

## Step 1: Load Real-World Data with Multicollinearity | الخطوة 1: تحميل بيانات العالم الحقيقي مع ارتباط متعدد

**BEFORE**: We need to learn regularization, but we need real data that has multicollinearity (correlated features).

**AFTER**: We'll load the California Housing dataset - real data where features are naturally correlated (multicollinearity) - this is when regularization helps most!

**Why California Housing?** This is REAL data from the 1990 census where features like AveRooms, AveBedrms, and Population are naturally correlated. When features are correlated, regular linear regression struggles. Ridge/Lasso handle this better!

In [ ]:
# Load real-world California Housing dataset
# This dataset naturally has multicollinearity (correlated features)
# Multicollinearity = features are correlated with each other
# This is a common real-world problem that regularization solves!

from sklearn.datasets import fetch_california_housing

print("\n📥 Loading California Housing dataset...")
print("تحميل مجموعة بيانات كاليفورنيا للإسكان...")

housing_data = fetch_california_housing()

# Create DataFrame
df = pd.DataFrame(housing_data.data, columns=housing_data.feature_names)
df['target'] = housing_data.target

print(f"\n✅ Real-world California Housing data loaded!")
print(f"   📊 This is REAL data from the 1990 California census")
print(f"   📈 Contains {len(df)} housing districts with {len(df.columns)-1} features")
print(f"\n🔍 Notice:")
print("   - Features like AveRooms, AveBedrms, Population are naturally correlated")
print("   - This multicollinearity makes regularization important!")
print("   - Ridge/Lasso will handle this better than regular linear regression")


In [ ]:
# Data summary comes first (showing what we loaded)
# Then we'll check for multicollinearity

print(f"\n📊 Real Data Summary:")
print(f"   Shape: {df.shape}")
print(f"   Features: {', '.join(housing_data.feature_names[:4])}... and more")
print(f"   Target: Median House Value (in $100,000s)")
print(f"\n📄 First 5 rows:")
print(df.head())
print("\n🔍 Notice:")
print("   - This is REAL data from 1990 California census")
print("   - Features are naturally correlated (multicollinearity)")
print("   - Regular regression may struggle; Ridge/Lasso will handle this better!")


In [ ]:
# Now check for multicollinearity in the loaded data
print("\n📊 Checking for multicollinearity in real data...")
print("   (Correlated features make regularization important)")

# Check correlation between features
correlation_matrix = df[housing_data.feature_names].corr()
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:  # High correlation threshold
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                corr_val
            ))

if high_corr_pairs:
    print(f"\n   ✅ Found {len(high_corr_pairs)} highly correlated feature pairs:")
    for feat1, feat2, corr in high_corr_pairs[:3]:  # Show first 3
        print(f"      - {feat1} ↔ {feat2}: {corr:.3f}")
    print("\n💡 What to Notice:")
    print("   - Correlation values close to 1.0 (or -1.0) mean features are highly correlated")
    print("   - Notice: AveRooms ↔ AveBedrms correlation = 0.848 (very high! They move together)")
    print("   - Notice: Latitude ↔ Longitude correlation = -0.925 (very high! They're related)")
    print("   - When features are correlated like this, regular Linear Regression can struggle")
    print("   - This is why we need Ridge/Lasso - they handle multicollinearity better! ✅")
else:
    print("   - Features have moderate correlation (still benefits from regularization)")

## Step 2: Prepare Data for Modeling | الخطوة 2: إعداد البيانات للنمذجة

**BEFORE**: We've loaded and explored the data, and we found multicollinearity (correlated features). Now we need to prepare the data for modeling!

**AFTER**: We'll split the data into training and test sets, and scale the features!

**Why scaling matters**: Regularization is sensitive to feature scale! Features on different scales (e.g., age vs income) will be penalized differently. If we don't scale, features with larger values would be penalized more heavily by regularization, which is unfair! We MUST scale first!

**What we'll do**:
1. Split data: 80% training, 20% testing (to evaluate models properly)
2. Scale features: Use StandardScaler (mean=0, std=1) so all features are on the same scale


In [ ]:
# Prepare features (X) and target (y) from real data
X_data = df[housing_data.feature_names]  # All 8 features
y_data = df['target']  # Median House Value

print(f"\n✅ Data prepared for modeling:")
print(f"   Features (X): {X_data.shape[1]} features")
print(f"   Target (y): {y_data.shape[0]} samples")
print(f"   Feature names: {', '.join(X_data.columns[:4])}... and more")

# Split data into training and testing sets
# train_test_split(X, y, test_size=0.2, random_state=42)
# - Splits data into training and testing sets
# - X: Features (input variables), y: Target (output variable)
# - test_size=0.2: 20% for testing, 80% for training
# - random_state=42: Seed for reproducibility (same split every time)
# - Returns: X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42
)

# Scale features (CRITICAL for regularization!)
# Regularization is sensitive to feature scale, so we MUST scale first!
# Without scaling, features with larger values would be penalized more
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("   ✅ Data split and scaled!")
print(f"   Training set: {X_train.shape[0]} samples")
print(f"   Test set: {X_test.shape[0]} samples")

## Understanding Alpha (α) - The Regularization Parameter | فهم Alpha (α) - معامل التنظيم

**BEFORE**: You know Ridge and Lasso use regularization, but what is alpha exactly?

**AFTER**: You'll understand that alpha controls how much regularization is applied!

**Why this matters**: Choosing the wrong alpha can lead to:
- **Overfitting** (alpha too small) → Model memorizes training data
- **Underfitting** (alpha too large) → Model becomes too simple, can't learn patterns

---

### 📚 What is Alpha (α)? | ما هو Alpha (α)؟

**Alpha (α)** is the **regularization strength parameter** that controls the trade-off between:
1. **Fitting the training data well** (low error on training data)
2. **Keeping the model simple** (small coefficients to prevent overfitting)

### 🔢 How Alpha Works | كيف يعمل Alpha

**The Math Behind It:**

For **Ridge Regression (L2)**, the cost function becomes:
```
Cost = MSE + α × (sum of squared coefficients)
```

For **Lasso Regression (L1)**, the cost function becomes:
```
Cost = MSE + α × (sum of absolute coefficients)
```

**Key Points:**
- **α = 0**: No regularization → Same as Linear Regression
- **α > 0**: Regularization is applied
- **Higher α** → More regularization → Smaller coefficients
- **Lower α** → Less regularization → Coefficients closer to Linear Regression

### 📊 Alpha Values and Their Effects | قيم Alpha وتأثيراتها

| Alpha Value | Effect | Result |
|-------------|--------|--------|
| **α = 0** | No regularization | Same as Linear Regression |
| **α = 0.01** | Very light regularization | Coefficients slightly shrunk |
| **α = 0.1** | Light regularization | Noticeable coefficient shrinkage |
| **α = 1.0** | Moderate regularization | Significant coefficient reduction |
| **α = 10.0** | Heavy regularization | Very small coefficients |
| **α = 100+** | Very heavy regularization | Most coefficients near zero (especially in Lasso) |

### 🎯 Alpha in Ridge vs Lasso | Alpha في Ridge مقابل Lasso

**Ridge (L2) Regularization:**
- **Shrinks coefficients smoothly** toward zero
- **Never sets coefficients to exactly zero**
- Keeps all features, just makes them smaller
- **Best for**: When all features might be relevant

**Lasso (L1) Regularization:**
- **Can set coefficients to exactly zero** (feature selection!)
- **More aggressive** shrinkage
- **Best for**: When you want to remove irrelevant features

### ⚖️ The Trade-off | المقايضة

```
Alpha (α) = 0          Alpha (α) = small        Alpha (α) = large
┌─────────────────┐   ┌─────────────────┐      ┌─────────────────┐
│ No Regularization│   │ Light Regularization │  │ Heavy Regularization│
│                 │   │                 │      │                 │
│ - Fits training │   │ - Balanced      │      │ - Very simple   │
│   data perfectly│   │ - Good          │      │ - May underfit  │
│ - May overfit!  │   │   generalization│      │ - Small coefs   │
│ - Large coefs   │   │ - Smaller coefs │      │ - Some = 0      │
│                 │   │                 │      │   (Lasso only)  │
└─────────────────┘   └─────────────────┘      └─────────────────┘
```

### 💡 How to Choose Alpha? | كيف تختار Alpha؟

1. **Start with a range** (e.g., 0.01, 0.1, 1.0, 10.0, 100.0)
2. **Train models** with different alpha values
3. **Evaluate on validation/test set** (not training set!)
4. **Choose the alpha** that gives the best test performance
5. **Use cross-validation** for more reliable selection (coming in Example 2!)

### 📝 Important Notes | ملاحظات مهمة

- **Alpha must be ≥ 0** (can't be negative)
- **Scaling matters!** Regularization requires scaled features (that's why we used StandardScaler)
- **Different datasets** need different alpha values
- **Ridge and Lasso** may have different optimal alpha values
- **Tuning alpha is a hyperparameter** search problem (we'll learn Grid Search in Unit 5!)

---

**In this notebook**, we'll try different alpha values and see how they affect model performance! 🚀


## Part 2: Building and Comparing Models | الجزء الثاني: بناء ومقارنة النماذج

**BEFORE**: We understand what alpha is and our data is prepared. Now let's build models!

**AFTER**: We'll build three models and compare them to see how regularization helps.

**Approach**: 
1. **Linear Regression** (baseline - no regularization)
2. **Ridge Regression** (L2 regularization - shrinks all coefficients)
3. **Lasso Regression** (L1 regularization - can remove features)

---

### Step 1: Linear Regression Baseline | الخطوة 1: الانحدار الخطي كخط أساس

**Why start here?** We need a baseline to compare against! Linear Regression has no regularization (α = 0), so we can see the difference when we add regularization.


In [ ]:
print("\n" + "=" * 60)
print("1. Linear Regression (Baseline)")
print("الانحدار الخطي (خط الأساس)")
print("=" * 60)

# Try regular linear regression first (no regularization)
# This is our baseline to compare against
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_pred = lr.predict(X_test_scaled)

lr_mse = mean_squared_error(y_test, lr_pred)
lr_r2 = r2_score(y_test, lr_pred)

print(f"\n📊 Linear Regression Results:")
print(f"   MSE: {lr_mse:.4f} (lower is better)")
print(f"   R² Score: {lr_r2:.4f} (closer to 1 is better)")
print(f"\n   Coefficients (first 5): {lr.coef_[:5]}")
print(f"   Notice: Some coefficients might be large or unstable")
print(f"\n💡 What we learned:")
print(f"   - Linear Regression baseline: MSE = {lr_mse:.4f}, R² = {lr_r2:.4f}")
print(f"   - Now let's see if Ridge or Lasso can improve this!")
print(f"   - Regularization should help stabilize these coefficients and potentially improve performance!")


### Step 2: Ridge Regression (L2 Regularization) | الخطوة 2: الانحدار ريدج (التنظيم L2)

**BEFORE**: We have our baseline. Now let's try Ridge regression with different alpha values.

**AFTER**: We'll see how different alpha values affect Ridge's performance and coefficients.

**What to expect**: 
- Ridge will try different alpha values (0.01, 0.1, 1.0, 10.0, 100.0)
- Remember from the Alpha explanation: higher alpha = more regularization = smaller coefficients
- Ridge keeps ALL features but shrinks them toward zero
- We'll find the best alpha that minimizes test MSE


In [ ]:
print("\n" + "=" * 60)
print("2. Ridge Regression (L2 Regularization)")
print("الانحدار ريدج (التنظيم L2)")
print("=" * 60)

# Ridge regression with different alpha values
# Alpha (α) = regularization strength parameter (see Alpha explanation section above!)
# - Higher alpha = more regularization = smaller coefficients
# - Lower alpha = less regularization = closer to Linear Regression
# Why try different alphas? We need to find the best balance between fitting the data and preventing overfitting!
# We'll evaluate each alpha on the test set and pick the one with lowest MSE
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
ridge_results = []

print("\n   Trying different alpha values...")
for alpha in alphas:
    ridge = Ridge(alpha=alpha)  # Create Ridge model with this alpha
    ridge.fit(X_train_scaled, y_train)  # Train on scaled data
    ridge_pred = ridge.predict(X_test_scaled)  # Predict on test data
    
    mse = mean_squared_error(y_test, ridge_pred)
    r2 = r2_score(y_test, ridge_pred)
    
    ridge_results.append({
        'alpha': alpha,
        'mse': mse,
        'r2': r2,
        'model': ridge
    })
    print(f"   Alpha {alpha:6.2f}: MSE = {mse:.4f}, R² = {r2:.4f}")

# Add interpretation after showing all results
print("\n💡 What to Notice:")
print("   - Look at how MSE and R² change as alpha increases")
print("   - Notice: Very low alpha (0.01) gives similar results to Linear Regression (MSE = 0.5559)")
print("   - Notice: The changes are SMALL (0.5559 → 0.5533) - but regularization still helps!")
print("   - Why small changes? The original model wasn't heavily overfitting, so regularization's benefit is modest")
print("   - Even small improvements (0.0026 lower MSE) can be meaningful - regularization stabilizes the model!")
print("   - Best alpha: 100.0 gives MSE = 0.5533 (lowest error, best performance)")
print("   - This teaches us: Regularization doesn't always dramatically improve performance, but it makes models more stable!")


### Step 3: Lasso Regression (L1 Regularization) | الخطوة 3: الانحدار لاسو (التنظيم L1)

**BEFORE**: We've seen how Ridge works. Now let's try Lasso!

**AFTER**: We'll see how Lasso differs from Ridge - it can actually remove features!

**Key difference from Ridge**:
- **Ridge**: Shrinks all coefficients toward zero (keeps all features)
- **Lasso**: Can set coefficients to EXACTLY zero (removes features automatically!)
- This is Lasso's special power: **automatic feature selection**
- We'll count how many features Lasso keeps vs removes


In [ ]:
print("\n" + "=" * 60)
print("3. Lasso Regression (L1 Regularization)")
print("الانحدار لاسو (التنظيم L1)")
print("=" * 60)

# Lasso regression with different alpha values
# Lasso does feature selection - sets some coefficients to zero!
lasso_results = []

print("\n   Trying different alpha values...")
for alpha in alphas:
    lasso = Lasso(alpha=alpha)  # Create Lasso model
    lasso.fit(X_train_scaled, y_train)  # Train on scaled data
    lasso_pred = lasso.predict(X_test_scaled)  # Predict
    
    mse = mean_squared_error(y_test, lasso_pred)
    r2 = r2_score(y_test, lasso_pred)
    
    # Count non-zero coefficients (features that Lasso kept)
    # Why check this? Lasso removes features by setting coefficients to zero!
    n_features = np.sum(np.abs(lasso.coef_) > 0.01)
    
    lasso_results.append({
        'alpha': alpha,
        'mse': mse,
        'r2': r2,
        'n_features': n_features,  # How many features Lasso kept
        'model': lasso
    })
    print(f"   Alpha {alpha:6.2f}: MSE = {mse:.4f}, R² = {r2:.4f}, Features = {n_features}/{len(X_data.columns)}")

# Add interpretation after showing all results
print("\n💡 What to Notice:")
print("   - Look at the 'Features' column: This shows how many features Lasso KEPT (non-zero coefficients)")
print("   - Notice: As alpha increases, fewer features are used (Lasso removes more features!)")
print("   - Notice: High alpha (1.0+) removes ALL features (Features = 0/8) - this is too much regularization!")
print("   - ⚠️ CRITICAL: When alpha ≥ 1.0, R² becomes NEGATIVE (R² = -0.0002)!")
print("     → Negative R² means the model is WORSE than just predicting the mean!")
print("     → This happens because Lasso removed ALL features (no model left!)")
print("     → This is what OVER-REGULARIZATION looks like - alpha is too high!")
print("   - Notice: Low alpha (0.01) keeps most features (Features = 7/8) - good balance")
print("   - Compare with Ridge: Ridge always uses ALL 8 features, Lasso can remove some!")
print("   - Which alpha performed best? Look for the lowest MSE value above! 👆")
print("   - Best alpha = 0.01 (lowest MSE = 0.5483, highest R² = 0.5816, keeps 7/8 features)")


In [ ]:
# Find best Ridge alpha (the one with lowest test MSE)
# We tried different alpha values above, now let's pick the best one
print("\n" + "=" * 60)
print("🔍 Finding Best Ridge Alpha")
print("العثور على أفضل قيمة Alpha لريج")
print("=" * 60)

best_ridge = min(ridge_results, key=lambda x: x['mse'])
print(f"\n✅ Best Ridge Model:")
print(f"   Alpha (α): {best_ridge['alpha']}")
print(f"   Test MSE: {best_ridge['mse']:.4f} (lowest error)")
print(f"   Test R²: {best_ridge['r2']:.4f} (higher is better)")
print(f"\n💡 Insight: Ridge with α={best_ridge['alpha']} gives the best performance!")

In [ ]:
# Find best Lasso alpha (the one with lowest test MSE)
# We tried different alpha values above, now let's pick the best one
print("\n" + "=" * 60)
print("🔍 Finding Best Lasso Alpha")
print("العثور على أفضل قيمة Alpha للاسو")
print("=" * 60)

best_lasso = min(lasso_results, key=lambda x: x['mse'])
print(f"\n✅ Best Lasso Model:")
print(f"   Alpha (α): {best_lasso['alpha']}")
print(f"   Test MSE: {best_lasso['mse']:.4f} (lowest error)")
print(f"   Test R²: {best_lasso['r2']:.4f} (higher is better)")
print(f"   Features used: {best_lasso['n_features']}/{len(X_data.columns)}")
print(f"\n💡 Insight: Lasso with α={best_lasso['alpha']} gives the best performance!")
if best_lasso['n_features'] < len(X_data.columns):
    print(f"   Plus: Lasso automatically removed {len(X_data.columns) - best_lasso['n_features']} features (feature selection!)")

### Step 4: Model Comparison | الخطوة 4: مقارنة النماذج

**BEFORE**: We've built three models (Linear, Ridge, Lasso) and found the best alpha for Ridge and Lasso. 

**AFTER**: Now let's compare all three models side-by-side to see which performs best!

**What we'll compare**:
- **Linear Regression**: Our baseline (no regularization, α = 0)
- **Best Ridge Model**: Best performing Ridge with optimal alpha
- **Best Lasso Model**: Best performing Lasso with optimal alpha

**Why compare?** This helps us understand:
- Does regularization actually help? (Ridge/Lasso vs Linear)
- Which regularization works better? (Ridge vs Lasso)
- What are the trade-offs? (performance vs simplicity)


In [ ]:
# 4. Comparison
print("\n" + "=" * 60)
print("4. Model Comparison")
print("مقارنة النماذج")
print("=" * 60)
# pd.DataFrame(data)
# - pd.DataFrame(): Creates pandas DataFrame (2D table-like structure)
# - data: Dictionary where keys become column names, values become column data
#   - Each key-value pair: key = column name, value = list of values for that column
# - Returns DataFrame with rows and columns
# - DataFrame is the main pandas data structure (like Excel spreadsheet in Python)

comparison = pd.DataFrame({
    'Model': ['Linear Regression', f'Ridge (α={best_ridge["alpha"]})',
              f'Lasso (α={best_lasso["alpha"]})'],
    'Test MSE': [lr_mse, best_ridge['mse'], best_lasso['mse']],
    'Test R²': [lr_r2, best_ridge['r2'], best_lasso['r2']]
})
print("\nComparison Table:")
print(comparison.to_string(index=False))

print("\n💡 Quick Look - What to Notice in the Table Above:")
print("   - Look at the Test MSE column: Which model has the LOWEST MSE? (Lower is better!)")
print("   - Look at the Test R² column: Which model has the HIGHEST R²? (Higher is better!)")
print("   - Notice: All three models have SIMILAR performance (values are close to each other)")
print("   - Notice: The differences are SMALL - this is normal and realistic!")
print("   - Notice: Lasso has the best MSE (0.5483) and R² (0.5816) - regularization helped!")
print("   - 💡 Learning point: Small improvements (0.0076 MSE difference) are meaningful in practice!")
print("   - Regularization's value isn't always huge performance gains - it's about stability and robustness!")

# Add interpretation
print("\n" + "=" * 60)
print("💡 Interpreting the Comparison | تفسير المقارنة")
print("=" * 60)

# Find best model
best_mse_idx = comparison['Test MSE'].idxmin()  # Lower MSE is better
best_r2_idx = comparison['Test R²'].idxmax()    # Higher R² is better (FIXED: was idxmin)
best_model_mse = comparison.loc[best_mse_idx, 'Model']
best_model_r2 = comparison.loc[best_r2_idx, 'Model']

print(f"\n📊 Best Model by MSE: {best_model_mse}")
print(f"   - Lowest error: {comparison['Test MSE'].min():.4f}")
print(f"   - This model has the smallest prediction errors")

print(f"\n📊 Best Model by R²: {best_model_r2}")
print(f"   - Highest R²: {comparison['Test R²'].max():.4f}")
print(f"   - This model explains the most variance")

print(f"\n🔍 Key Observations:")
mse_diff = comparison['Test MSE'].max() - comparison['Test MSE'].min()
r2_diff = comparison['Test R²'].max() - comparison['Test R²'].min()

if mse_diff < 0.01:
    print(f"   - ✅ All models perform similarly (MSE difference: {mse_diff:.4f})")
    print(f"   - Regularization shows SMALL improvements - this is actually normal and realistic!")
    print(f"   - Why small changes? The original Linear Regression model wasn't heavily overfitting")
    print(f"   - This teaches us: Regularization doesn't always create dramatic improvements")
    print(f"   - But regularization still helps: It stabilizes coefficients and prevents overfitting!")
    print(f"   - Even small improvements matter in real-world applications")
else:
    print(f"   - ⚠️  Significant performance difference (MSE range: {mse_diff:.4f})")
    print(f"   - Regularization {'improved' if best_model_mse != 'Linear Regression' else 'did not improve'} performance")
    print(f"   - This suggests the original model had overfitting issues that regularization solved!")

if r2_diff < 0.01:
    print(f"   - ✅ R² scores are very close (difference: {r2_diff:.4f})")
else:
    print(f"   - ⚠️  R² scores differ (range: {r2_diff:.4f})")

print(f"\n📚 What This Teaches Us:")
print(f"   - Compare models using multiple metrics (MSE and R²)")
print(f"   - Lower MSE = better predictions (less error)")
print(f"   - Higher R² = better fit (explains more variance)")
print(f"   - Regularization (Ridge/Lasso) helps when there's overfitting")
print(f"   - ⚠️ Important: Small improvements (like 0.0076 MSE difference) are NORMAL and REALISTIC!")
print(f"   - Not every dataset will show dramatic improvements - that's okay!")
print(f"   - Regularization's value: It stabilizes models and prevents overfitting, even if performance gains are modest")
print(f"   - Lasso has advantage: automatic feature selection (removes irrelevant features)")
print(f"   - In practice, regularization is about robustness and generalization, not just performance numbers")

# Check if Lasso removed features
if best_lasso['n_features'] < len(X_data.columns):
    print(f"\n💡 Lasso Feature Selection:")
    print(f"   - Lasso used only {best_lasso['n_features']}/{len(X_data.columns)} features")
    print(f"   - Removed {len(X_data.columns) - best_lasso['n_features']} features (set coefficients to 0)")
    print(f"   - This is Lasso's unique advantage: automatic feature selection!")
    print(f"   - Simpler model (fewer features) = easier to interpret")

In [ ]:
# 5. Coefficient Comparison
print("\n" + "=" * 60)
print("5. Coefficient Comparison")
print("مقارنة المعاملات")
print("=" * 60)
# pd.DataFrame(data)
# - pd.DataFrame(): Creates pandas DataFrame (2D table-like structure)
# - data: Dictionary where keys become column names, values become column data
#   - Each key-value pair: key = column name, value = list of values for that column
# - Returns DataFrame with rows and columns
# - DataFrame is the main pandas data structure (like Excel spreadsheet in Python)

coef_comparison = pd.DataFrame({
    'Feature': X_data.columns,
    'Linear': lr.coef_,
    'Ridge': best_ridge['model'].coef_,
    'Lasso': best_lasso['model'].coef_
})
print("\nCoefficient Comparison (first 5 features):")
print(coef_comparison.head().to_string(index=False))

print("\n💡 Quick Look - What to Notice in the Table Above:")
print("   - Look at the 'Population' row: Linear= -0.002308, Ridge= -0.000024, Lasso= -0.000000")
print("   - Notice: Lasso set Population coefficient to EXACTLY 0.000000 (removed this feature!)")
print("   - Notice: Ridge shrunk it to -0.000024 (very small, but NOT zero - still keeps the feature)")
print("   - Look at other features: Compare how Ridge shrinks vs Lasso removes coefficients")
print("   - This shows Lasso's feature selection power - it can completely remove features! 🎯")
print("\nLasso shrinks many coefficients to zero (feature selection)")
print("لاسو يقلص العديد من المعاملات إلى الصفر (اختيار الميزات)")

# Add interpretation
print("\n" + "=" * 60)
print("💡 Understanding Coefficient Differences | فهم اختلافات المعاملات")
print("=" * 60)

# Count zero coefficients
lasso_zeros = (np.abs(best_lasso['model'].coef_) < 0.01).sum()
ridge_zeros = (np.abs(best_ridge['model'].coef_) < 0.01).sum()
linear_zeros = (np.abs(lr.coef_) < 0.01).sum()

print(f"\n📊 Coefficient Analysis:")
print(f"   - Linear Regression: {linear_zeros} coefficients near zero")
print(f"   - Ridge: {ridge_zeros} coefficients near zero (shrinks but keeps all)")
print(f"   - Lasso: {lasso_zeros} coefficients set to zero (feature selection!)")

print(f"\n🔍 What This Shows:")
print(f"   - Ridge: Shrinks coefficients toward 0 but keeps all features")
print(f"   - Lasso: Can completely remove features (coefficient = 0)")
print(f"   - Lasso's sparsity: Only {best_lasso['n_features']} features have non-zero coefficients")

# Compare coefficient magnitudes
coef_diff_ridge = np.abs(lr.coef_ - best_ridge['model'].coef_)
coef_diff_lasso = np.abs(lr.coef_ - best_lasso['model'].coef_)

print(f"\n📊 Coefficient Shrinking:")
print(f"   - Ridge average change: {coef_diff_ridge.mean():.4f}")
print(f"   - Lasso average change: {coef_diff_lasso.mean():.4f}")
if coef_diff_lasso.mean() > coef_diff_ridge.mean():
    print(f"   - Lasso shrinks coefficients more aggressively")

print(f"\n📚 What This Teaches Us:")
print(f"   - Regularization reduces coefficient magnitudes (prevents overfitting)")
print(f"   - Ridge: Gentle shrinking, keeps all features")
print(f"   - Lasso: Aggressive shrinking, removes irrelevant features")
print(f"   - Smaller coefficients = simpler model = better generalization")
print(f"   - Lasso is useful when you have many features (automatic feature selection)")
print(f"   - Ridge is useful when all features might be relevant")

### Step 5: Coefficient Comparison | الخطوة 5: مقارنة المعاملات

**BEFORE**: We've compared model performance (MSE, R²). But what's happening "under the hood" with the coefficients?

**AFTER**: We'll see how Ridge and Lasso actually change the coefficients compared to Linear Regression!

**Why compare coefficients?**
- **Understand regularization's effect**: How does it shrink coefficients?
- **See Ridge vs Lasso difference**: Ridge shrinks smoothly, Lasso can set to zero
- **Verify feature selection**: Lasso should have some coefficients exactly equal to zero
- **Better intuition**: Seeing coefficient changes helps understand why regularization works!


## Step 7: Decision Framework - When to Use Ridge vs Lasso | الخطوة 7: إطار القرار - متى تستخدم ريدج مقابل لاسو

**BEFORE**: You've learned how to build Ridge and Lasso models, but when should you use each one?

**AFTER**: You'll have a clear decision framework to choose between Ridge, Lasso, or regular Linear Regression!

**Why this matters**: Using the wrong regularization method can:
- **Miss important features** → Lasso removes features you need
- **Keep irrelevant features** → Ridge keeps all features even when some are noise
- **Poor performance** → Wrong method leads to worse predictions

---

### 🎯 Decision Framework: Ridge vs Lasso vs Linear Regression | إطار القرار: ريدج مقابل لاسو مقابل الانحدار الخطي

**Key Question**: Should I use **LINEAR REGRESSION**, **RIDGE**, or **LASSO**?

#### Decision Tree:

```
Do you have overfitting?
├─ NO → Use LINEAR REGRESSION ✅
│   └─ Why? No need for regularization if model generalizes well
│
└─ YES → Check your situation:
    ├─ Many features (>20)? → Continue to next step
    │
    ├─ Need feature selection? → Use LASSO ✅
    │   └─ Why? Lasso removes irrelevant features automatically
    │
    ├─ Multicollinearity present? → Use RIDGE ✅
    │   └─ Why? Ridge handles correlated features better
    │
    ├─ All features important? → Use RIDGE ✅
    │   └─ Why? Ridge keeps all features, just shrinks them
    │
    └─ Want interpretability? → Use LASSO ✅
        └─ Why? Fewer features = simpler model
```

#### Detailed Decision Process:

```
Step 1: Check if regularization is needed
├─ Train R² >> Test R²? → YES, overfitting present
│   └─ Use Ridge or Lasso
│
└─ Train R² ≈ Test R²? → NO, no overfitting
    └─ Use Linear Regression (simpler)

Step 2: If overfitting, choose regularization type
├─ Do you have many features (>20)?
│   ├─ YES → Continue to step 3
│   └─ NO → Try Ridge first (simpler)
│
├─ Do you need feature selection?
│   ├─ YES → Use LASSO
│   │   └─ Why? Automatically removes irrelevant features
│   └─ NO → Continue to step 4
│
├─ Is there multicollinearity (correlated features)?
│   ├─ YES → Use RIDGE
│   │   └─ Why? Ridge handles correlations better
│   └─ NO → Continue to step 5
│
└─ Are all features potentially important?
    ├─ YES → Use RIDGE
    │   └─ Why? Keeps all features, just shrinks them
    └─ NO → Use LASSO
        └─ Why? Removes irrelevant features
```

---

### 📊 Comparison Table: Linear vs Ridge vs Lasso | جدول المقارنة

| Method | When to Use | Pros | Cons | Example |
|--------|-------------|------|------|---------|
| **Linear Regression** | No overfitting, few features, interpretable | • Simple<br>• Fast<br>• Interpretable<br>• No hyperparameters | • Can overfit<br>• Sensitive to outliers<br>• Can't handle many features | Small dataset, < 10 features |
| **Ridge (L2)** | Overfitting, multicollinearity, all features important | • Prevents overfitting<br>• Handles multicollinearity<br>• Keeps all features<br>• Stable | • Doesn't remove features<br>• All features contribute<br>• Less interpretable | Many correlated features, all potentially important |
| **Lasso (L1)** | Overfitting, many features, need feature selection | • Prevents overfitting<br>• Automatic feature selection<br>• More interpretable<br>• Simpler models | • Can remove important features<br>• Unstable with correlated features<br>• May over-regularize | High-dimensional data, feature selection needed |

---

### ✅ When to Use Each Method | متى تستخدم كل طريقة

#### Use Linear Regression when:
1. **No Overfitting** ✅
   - Train and test performance are similar
   - Model generalizes well
   - **Example**: Small dataset, few features, good performance

2. **Few Features** ✅
   - Less than 10-15 features
   - All features are important
   - **Example**: House price from size, bedrooms, age

3. **Interpretability Critical** ✅
   - Need to understand exact coefficients
   - No regularization complexity needed
   - **Example**: Medical diagnosis, regulatory compliance

#### Use Ridge Regression when:
1. **Overfitting Present** ✅
   - Train R² much higher than test R²
   - Model memorizes training data
   - **Example**: Polynomial regression with high degree

2. **Multicollinearity** ✅
   - Features are highly correlated
   - Ridge handles correlations better than Lasso
   - **Example**: House features (size, rooms, area all correlated)

3. **All Features Important** ✅
   - Don't want to remove any features
   - Just want to shrink coefficients
   - **Example**: All features are domain-relevant

4. **Many Features** ✅
   - 20+ features
   - Need regularization but want to keep all features
   - **Example**: 50+ features from feature engineering

#### Use Lasso Regression when:
1. **Feature Selection Needed** ✅
   - Many features, some are noise
   - Want automatic feature selection
   - **Example**: 100+ features, need to find important ones

2. **Sparse Solution** ✅
   - Expect only few features matter
   - Want interpretable model
   - **Example**: Gene expression data (few genes matter)

3. **High-Dimensional Data** ✅
   - More features than samples
   - Need to reduce dimensionality
   - **Example**: Text data with thousands of features

---

### ❌ When NOT to Use Each Method | متى لا تستخدم كل طريقة

#### Don't use Linear Regression when:
1. **Severe Overfitting** ❌
   - Train R² >> Test R²
   - **Use Instead**: Ridge or Lasso

2. **Many Features** ❌
   - 50+ features
   - **Use Instead**: Ridge or Lasso

3. **Multicollinearity** ❌
   - Highly correlated features
   - **Use Instead**: Ridge

#### Don't use Ridge when:
1. **Feature Selection Needed** ❌
   - Want to remove irrelevant features
   - **Use Instead**: Lasso

2. **Sparse Solution Expected** ❌
   - Only few features matter
   - **Use Instead**: Lasso

#### Don't use Lasso when:
1. **Multicollinearity Present** ❌
   - Features are highly correlated
   - Lasso may randomly select one
   - **Use Instead**: Ridge

2. **All Features Important** ❌
   - Don't want to remove any features
   - **Use Instead**: Ridge

3. **More Features than Samples** ❌
   - Lasso can select at most n features (n = samples)
   - **Use Instead**: Ridge or Elastic Net

---

### 📊 Real-World Examples | أمثلة من العالم الحقيقي

#### Example 1: House Price Prediction (10 features) ✅ LINEAR REGRESSION
- **Features**: Size, bedrooms, age, location, etc. (10 total)
- **Overfitting**: No (train R² = 0.85, test R² = 0.83)
- **Decision**: ✅ Use Linear Regression
- **Reasoning**: No overfitting, few features, all important

#### Example 2: House Price Prediction (50 features) ✅ RIDGE
- **Features**: Size, bedrooms, age, location, neighborhood stats, etc. (50 total)
- **Overfitting**: Yes (train R² = 0.95, test R² = 0.75)
- **Multicollinearity**: Yes (size, rooms, area all correlated)
- **Decision**: ✅ Use Ridge Regression
- **Reasoning**: Overfitting, many features, multicollinearity, all features potentially important

#### Example 3: Gene Expression Analysis (1000 features) ✅ LASSO
- **Features**: 1000 genes, only 10-20 matter
- **Overfitting**: Yes (train R² = 0.98, test R² = 0.60)
- **Feature Selection**: Critical (need to find important genes)
- **Decision**: ✅ Use Lasso Regression
- **Reasoning**: Many features, need feature selection, sparse solution expected

#### Example 4: Sales Prediction (30 features, some noise) ⚠️ TRY BOTH
- **Features**: 30 features, some are noise
- **Overfitting**: Yes (train R² = 0.92, test R² = 0.78)
- **Decision**: ⚠️ Try both Ridge and Lasso, compare
- **Reasoning**: Overfitting present, some features may be noise, try both methods

---

### ✅ Key Takeaways | النقاط الرئيسية

1. **Start with Linear Regression** - Always try simplest model first
2. **Check for overfitting** - Compare train vs test performance
3. **Ridge for multicollinearity** - When features are correlated
4. **Lasso for feature selection** - When you need to remove features
5. **Tune alpha** - Critical hyperparameter for both methods
6. **Scale features first** - Regularization requires scaled features
7. **Try both** - Sometimes try Ridge and Lasso, pick the best

---

### 🎓 Practice Decision-Making | ممارسة اتخاذ القرار

**Scenario 1**: Predicting house prices with 8 features (size, bedrooms, age, etc.)
- **Overfitting**: No (train R² = 0.88, test R² = 0.86)
- **Decision**: ✅ Linear Regression (no overfitting, few features)

**Scenario 2**: Predicting sales with 50 features, many correlated
- **Overfitting**: Yes (train R² = 0.94, test R² = 0.76)
- **Multicollinearity**: Yes (many correlated features)
- **Decision**: ✅ Ridge Regression (overfitting, multicollinearity, all features important)

**Scenario 3**: Predicting disease from 500 gene expressions
- **Overfitting**: Yes (train R² = 0.97, test R² = 0.65)
- **Feature Selection**: Critical (only few genes matter)
- **Decision**: ✅ Lasso Regression (many features, need feature selection, sparse solution)

---

**Connection to Next Steps**: 
- 📓 **Example 2: Cross-Validation** - For proper evaluation of Ridge/Lasso models
- 📓 **Unit 3: Classification** - Same regularization concepts apply to classification
- 📓 **Unit 5, Example 1: Grid Search** - For tuning alpha hyperparameter


### Step 6: Visualization | الخطوة 6: التصور

**BEFORE**: We've compared models and coefficients using numbers. Now let's visualize the results!

**AFTER**: We'll create plots showing:
1. How alpha affects model performance (MSE vs Alpha)
2. How coefficients differ between models (coefficient magnitudes)

**Why visualize?**
- **See patterns visually**: Graphs make it easier to understand the relationships
- **Compare at a glance**: Visual comparison is often clearer than numbers
- **Alpha effect**: See how different alpha values affect Ridge and Lasso
- **Coefficient differences**: Visualize how Ridge shrinks vs Lasso removes coefficients


In [ ]:
# 6. Visualization
print("\n" + "=" * 60)
print("6. Visualization")
print("التصور")
print("=" * 60)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Alpha vs MSE
axes[0].semilogx([r['alpha'] for r in ridge_results],
                 [r['mse'] for r in ridge_results],
                 'o-', label='Ridge', linewidth=2)
axes[0].semilogx([l['alpha'] for l in lasso_results],
                 [l['mse'] for l in lasso_results],
                 's-', label='Lasso', linewidth=2)
axes[0].axhline(lr_mse, color='r', linestyle='--', label='Linear Regression')
axes[0].set_xlabel('Alpha (Regularization Strength)')
axes[0].set_ylabel('Test MSE')
axes[0].set_title('Regularization vs Model Performance')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Coefficient magnitudes
n_features = len(lr.coef_)  # Get actual number of features from the model
axes[1].bar(range(n_features), np.abs(lr.coef_), alpha=0.7, label='Linear', width=0.25)
axes[1].bar([i + 0.25 for i in range(n_features)], np.abs(best_ridge['model'].coef_),
            alpha=0.7, label='Ridge', width=0.25)
axes[1].bar([i + 0.5 for i in range(n_features)], np.abs(best_lasso['model'].coef_),
            alpha=0.7, label='Lasso', width=0.25)
axes[1].set_xlabel('Feature Index')
axes[1].set_ylabel('Absolute Coefficient Value')
axes[1].set_title('Coefficient Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('ridge_lasso_comparison.png', dpi=300, bbox_inches='tight')
print("\n✓ Plot saved as 'ridge_lasso_comparison.png'")
plt.show()
print("\n" + "=" * 60)
print("Example 1 Complete! ✓")
print("اكتمل المثال 1! ✓")
print("=" * 60)